# Report on the Neural Network Model

## Overview of the analysis 

This analysis explains my attempts to construct a deep learning neural network to predict which applicants will be successful with funding, using the Alphabet Soup dataset.  

## Results: 

**Data Preprocessing**

- Target variable: IS_SUCCESSFUL 

- Feature variables: APPLICATION_TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION, STATUS, INCOME_AMT, SPECIAL_CONSIDERATIONS, ASK_AMT

- Useless variables: EIN, NAME

**Compiling, Training, and Evaluating the Model**

On my first attempt, my preprocessed data had 43 inputs. I created a neural network with input_dim of 43, 2 hidden layers with 100 units each, and 10 epochs. I chose 100 units as a value because it is advisable to start with a number 2 to 3 times as big as the input dimension. 2 layers and 10 epochs were a modest and sensible starting point for benchmarking. Relu was the activation function for all of the layers except the last one, which used sigmoid. 

My best performance with these specifications was: 

- **Loss: 0.5517758131027222, Accuracy: 0.729912519454956**

While decent, this result did not meet our desired threshold of 75% accuracy.  To address this, I fed the keras-tuner in Google Colaboratory all of the same specifications from above, hoping for a promising lead on the best model. Its output for the best model could not be replicated with the specifications it suggested, but they led me to experiment with similar settings. I ended up running 100 epochs with an input layer of 9 units in relu activation, and 2 hidden layers of 9 units, one in relu and one in tanh activation. The output layer used the sigmoid function.  With these parameters, I was able to achieve this performance:

- **Loss: 0.5544543266296387, Accuracy: 0.7315452098846436**

**Reflections**

At this point I was at a loss as to how to improve this model's performance.  I went back to the drawing board ... which of these "features" were really contributing to the IS_SUCCESSFUL values?  


How many neurons, layers, and activation functions did you select for your neural network model, and why?
Were you able to achieve the target model performance?
What steps did you take in your attempts to increase model performance?

**Summary**: Summarize the overall results of the deep learning model. Include a recommendation for how a different model could solve this classification problem, and then explain your recommendation.